# Importing the necessary libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from zipfile import ZipFile
# !pip install plotly
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import KMeans
import plotnine as pn
import plydata.cat_tools as cat
# machine learning
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# Importing the lifetimes package
import lifetimes
from sklearn.metrics import mean_absolute_error

# Loading the Data

In [2]:
# Laoding the data into a dataframe.
CDNOW_DATA = pd.read_csv("Data/CDNOW_master.txt", sep='\s+')
CDNOW_DATA.shape

(69658, 4)

In [3]:
CDNOW_DATA.head() # Looking at the top 5 rows of the dataframe.

,00001,19970101,1,11.77
0,2,19970112,1,12.00
1,2,19970112,5,77.00
2,3,19970102,2,20.76
3,3,19970330,2,20.76
4,3,19970402,2,19.54


# Data Cleaning

In [4]:
CDNOW_DATA.info() # info() gives us the number of non null values and the data type of each column.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69658 entries, 0 to 69657
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   00001     69658 non-null  int64  
 1   19970101  69658 non-null  int64  
 2   1         69658 non-null  int64  
 3   11.77     69658 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 2.1 MB


In [5]:
CDNOW_DATA.columns = ["CustomerID", "TransactionDate", "UnitsSold", "Price"] # Assigning the column names to the data frame.

In [6]:
CDNOW_DATA["CustomerID"] = CDNOW_DATA["CustomerID"].astype(str) # changing the datatype of the CustomerID to string.

In [7]:
# changing the datatye of the TransactionDate column to datetime.
CDNOW_DATA['TransactionDate'] = CDNOW_DATA['TransactionDate'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d'))

In [8]:
CDNOW_DATA.head()  # Looking at the top 5 rows of the dataframe.

,CustomerID,TransactionDate,UnitsSold,Price
0,2,1997-01-12,1,12.00
1,2,1997-01-12,5,77.00
2,3,1997-01-02,2,20.76
3,3,1997-03-30,2,20.76
4,3,1997-04-02,2,19.54


# CLV Mathematically

In [9]:
# Creating a customer level dataframe
customer_level_data = CDNOW_DATA.groupby("CustomerID").aggregate({"TransactionDate": lambda x: (x.max() - x.min()).days,
                                                              "CustomerID": lambda x: len(x),
                                                   "Price" : lambda x: x.sum()})
customer_level_data.columns = ['num_days', 'num_transactions', 'spent_money']
customer_level_data.head()

,num_days,num_transactions,spent_money
CustomerID,,,
10,0,1,39.31
100,474,3,55.24
1000,0,1,8.99
10000,175,3,77.42
10001,245,2,19.78


### AVG_ORDER_VALUE

In [10]:
#Calculating the average order value for each customer
customer_level_data['avg_order_value']=customer_level_data['spent_money']/customer_level_data['num_transactions']
customer_level_data.head()

,num_days,num_transactions,spent_money,avg_order_value
CustomerID,,,,
10,0,1,39.31,39.310000
100,474,3,55.24,18.413333
1000,0,1,8.99,8.990000
10000,175,3,77.42,25.806667
10001,245,2,19.78,9.890000


### PURCHASE FREQUNECY


In [11]:
purchase_frequency=sum(customer_level_data['num_transactions'])/customer_level_data.shape[0]
purchase_frequency

2.9554923840638128

### REPEAT RATE

In [12]:
# Repeat Rate
repeat_rate=customer_level_data[customer_level_data.num_transactions > 1].shape[0]/customer_level_data.shape[0]
repeat_rate

0.49480249480249483

### CHURN

In [13]:
#Churn Rate
churn_rate=1-repeat_rate
churn_rate

0.5051975051975052

In [14]:
purchase_frequency,repeat_rate,churn_rate

(2.9554923840638128, 0.49480249480249483, 0.5051975051975052)

### PROFIT

In [15]:
# Assuming profit marging to be of 5%
customer_level_data["profit"] = customer_level_data["spent_money"]* 0.05
customer_level_data.head()

,num_days,num_transactions,spent_money,avg_order_value,profit
CustomerID,,,,,
10,0,1,39.31,39.310000,1.9655
100,474,3,55.24,18.413333,2.7620
1000,0,1,8.99,8.990000,0.4495
10000,175,3,77.42,25.806667,3.8710
10001,245,2,19.78,9.890000,0.9890


### CLV

In [16]:
# CLV
customer_level_data['CLV']=(customer_level_data['avg_order_value']*purchase_frequency)/churn_rate

### CLTV

In [17]:
customer_level_data['cust_lifetime_value']=customer_level_data['CLV']*customer_level_data['profit']
customer_level_data.head()

,num_days,num_transactions,spent_money,avg_order_value,profit,CLV,cust_lifetime_value
CustomerID,,,,,,,
10,0,1,39.31,39.310000,1.9655,229.970268,452.006562
100,474,3,55.24,18.413333,2.7620,107.721170,297.525872
1000,0,1,8.99,8.990000,0.4495,52.593048,23.640575
10000,175,3,77.42,25.806667,3.8710,150.973443,584.418198
10001,245,2,19.78,9.890000,0.9890,57.858203,57.221763


In [18]:
cdnow = CDNOW_DATA.copy()

cdnow['month_yr'] = cdnow['TransactionDate'].apply(lambda x: x.strftime('%b-%Y'))
cdnow.head()

,CustomerID,TransactionDate,UnitsSold,Price,month_yr
0,2,1997-01-12,1,12.00,Jan-1997
1,2,1997-01-12,5,77.00,Jan-1997
2,3,1997-01-02,2,20.76,Jan-1997
3,3,1997-03-30,2,20.76,Mar-1997
4,3,1997-04-02,2,19.54,Apr-1997


In [19]:
sale=cdnow.pivot_table(index=['CustomerID'],columns=['month_yr'],values='Price',aggfunc='sum',fill_value=0).reset_index()
sale.head()


month_yr,CustomerID,Apr-1997,Apr-1998,Aug-1997,Dec-1997,Feb-1997,Feb-1998,Jan-1997,Jan-1998,Jul-1997,Jun-1997,Jun-1998,Mar-1997,Mar-1998,May-1997,May-1998,Nov-1997,Oct-1997,Sep-1997
0,10,0.0,0.00,0.0,0.00,0.00,0.0,39.31,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
1,100,0.0,28.98,0.0,12.49,0.00,0.0,13.77,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
2,1000,0.0,0.00,0.0,0.00,0.00,0.0,8.99,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
3,10000,0.0,0.00,0.0,0.00,14.37,0.0,0.00,0.0,22.54,40.51,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
4,10001,0.0,0.00,0.0,0.00,6.79,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,12.99,0.0


In [20]:
sale['CLV']=sale.iloc[:,2:].sum(axis=1)
sale.head()

month_yr,CustomerID,Apr-1997,Apr-1998,Aug-1997,Dec-1997,Feb-1997,Feb-1998,Jan-1997,Jan-1998,Jul-1997,Jun-1997,Jun-1998,Mar-1997,Mar-1998,May-1997,May-1998,Nov-1997,Oct-1997,Sep-1997,CLV
0,10,0.0,0.00,0.0,0.00,0.00,0.0,39.31,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,39.31
1,100,0.0,28.98,0.0,12.49,0.00,0.0,13.77,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,55.24
2,1000,0.0,0.00,0.0,0.00,0.00,0.0,8.99,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,8.99
3,10000,0.0,0.00,0.0,0.00,14.37,0.0,0.00,0.0,22.54,40.51,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,77.42
4,10001,0.0,0.00,0.0,0.00,6.79,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,12.99,0.0,19.78


In [21]:
cdnow["TransactionDate"].max()

Timestamp('1998-06-30 00:00:00')

# CLV prediction based on the latest 6 month sales data:

>Predictive modeling on the customer lifetime value instead of the sales expected in next 90 days

In [22]:
# Selecting features as the latest 6 month data

X2=sale[['Jan-1998','Feb-1998', 'Mar-1998','Apr-1998','May-1998','Jun-1998']]
y=sale[['CLV']]

### XGBOOST

In [23]:
xgb = XGBRegressor(objective='reg:squarederror', random_state = 123) #model

In [24]:
# GridSearch with all the hyperparameters
xgb =GridSearchCV(estimator=xgb, param_grid = dict(learning_rate = [0.01, 0.1, 0.3, 0.5]), scoring='neg_mean_absolute_error', refit= True, cv = 5)

In [25]:
xgb.fit(X2, y) # Fitting the data

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    enable_categorical=False, gamma=None,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, predictor=None,
                                    random_state=123, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
        

In [26]:
xgb.best_score_ # NAE for the best model

-57.91302282560919

In [27]:
xgb.best_params_ # Hyper parameters for the best model

{'learning_rate': 0.01}

In [28]:
predictions_xgb_reg_6model = xgb.predict(X2) # prediction

In [29]:
XGBOOST_6M_REG = pd.DataFrame({"prediction_xgb_6m" : predictions_xgb_reg_6model.tolist()}).merge(X2,left_index=True, right_index=True).merge(sale["CustomerID"],left_index=True, right_index=True)

In [30]:
XGBOOST_6M_REG.to_csv("XGB_reg_6m_prediction.csv") # Prediction to csv.

### Linear Regression

In [31]:
lr = LinearRegression()

In [32]:
parameters = {'fit_intercept':[True,False],  'copy_X':[True, False]} # Hyperparameters
lr_reg_model = GridSearchCV(lr,parameters, cv=5, scoring='neg_mean_absolute_error')
lr_reg_model.fit(X2, y) #fittig the data

GridSearchCV(cv=5, estimator=LinearRegression(),
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False]},
             scoring='neg_mean_absolute_error')

In [33]:
lr_reg_model.best_score_ # NAE for the best model

-57.00890695623277

In [34]:
lr_reg_model.best_params_ # Hyperparameters for the best model.

{'copy_X': True, 'fit_intercept': True}

In [35]:
predictions_lr_reg_6model = lr_reg_model.predict(X2).reshape(-1)
predictions_lr_reg_6model

array([ 53.33973479, 117.48789003,  53.33973479, ...,  53.33973479,
        53.33973479,  53.33973479])

In [36]:
# Storing the prediction in a dataframe.
LinearReg_6M_REG = pd.DataFrame({"prediction_lr_6m" : predictions_lr_reg_6model.tolist()}).merge(X2,left_index=True, right_index=True).merge(sale["CustomerID"],left_index=True, right_index=True)

In [37]:
LinearReg_6M_REG.to_csv("lr_reg_6m_prediction.csv")# Prediction to csv.

### Random Forest Regression

In [38]:
# Hyper Parameters
param_grid = dict(n_estimators = [10, 20, 50, 100, 500, 1000], max_depth = [2, 5], max_features = ['auto', 'sqrt', 'log2'])
rf = RandomForestRegressor() # Model
rf_regressor =GridSearchCV(estimator=rf, param_grid = param_grid, scoring='neg_mean_absolute_error', refit= True, cv = 5) 

In [39]:
rf_regressor.fit(X2, y.values.ravel()) # fitting the data

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 5],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 20, 50, 100, 500, 1000]},
             scoring='neg_mean_absolute_error')

In [40]:
rf_regressor.best_score_ # NAE for the best model

-61.98302547573312

In [41]:
predictions_rf_reg_6model = rf_regressor.predict(X2) # Prediction

In [42]:
RFReg_6M_REG = pd.DataFrame({"prediction_rf_6m" : predictions_rf_reg_6model.tolist()}).merge(X2,left_index=True, right_index=True).merge(sale["CustomerID"],left_index=True, right_index=True)

In [43]:
RFReg_6M_REG.to_csv("rf_reg_6m_prediction.csv")# Prediction to csv.